In [1]:
import numpy as np
import pandas as pd
import zipfile
import pandas as pd
import os
import json
import re
import ast
import multiprocessing
import copy
import ast

In [4]:
cpa_statistics = pd.read_csv('output-data/cpa_statistics.csv')
cpa_statistics = cpa_statistics.loc[cpa_statistics['column_count'] >= 50]
cpa_statistics

,cpa_label,table_count,column_count,tables_and_densities
0,url,228671,228671,{'Product_corememoriesco.com_September2020.jso...
1,name,947672,947672,{'Product_corememoriesco.com_September2020.jso...
2,priceCurrency,774782,774782,{'Product_corememoriesco.com_September2020.jso...
3,price,728024,728024,{'Product_corememoriesco.com_September2020.jso...
4,description,776148,778315,{'Product_corememoriesco.com_September2020.jso...
...,...,...,...,...
226,publication,87,87,{'Book_montellolibrary.org_September2020.json....
227,comment,64,64,{'CreativeWork_fortrader.org_September2020.jso...
229,seasonNumber,111,111,{'TVEpisode_animecons.tv_September2020.json.gz...
233,availabilityEnds,79,79,{'Product_civico.com_September2020.json.gz': {...


In [5]:
rels = pd.read_csv('data/Final CTA and CPA Labels.csv')
rels = rels.loc[rels['CPA label'].isin(cpa_statistics['cpa_label'].tolist())]
rels

,class,column_name,CPA label,CTA label,table_number,fallback_CTA_label
1,Product,offers,offers,Offer,782333,NaN
2,Product,offers:pricecurrency,priceCurrency,currency,763066,NaN
3,Product,offers:price,price,price,716385,NaN
4,Product,offers:availability,availability,ItemAvailability,432124,NaN
5,Product,offers:pricevaliduntil,priceValidUntil,Date,170373,NaN
...,...,...,...,...,...,...
667,Museum,telephone,telephone,telephone,160,NaN
668,Museum,geo,geo,"GeoCoordinates, GeoShape",158,geo
669,Museum,geo:latitude,latitude,CoordinateAT,158,NaN
670,Museum,geo:longitude,longitude,CoordinateAT,158,NaN


In [95]:
#CPA labels: column name to its CPA label
rel_lbls = {}
for index, row in rels.iterrows():
    rel_lbls[row['column_name']] = row['CPA label']
rel_lbls['name'] = 'name'

In [10]:
annotated_tables = pd.read_csv('output-data/statistics/expanded_tables_annotations_cpa.csv')
annotated_tables = annotated_tables.loc[(annotated_tables['column_count'] >= 3)]
annotated_tables

,class,file_name,column_count,number_of_rows,overall_table_density,all_cols,rel_labels,type_labels
0,Product,Product_corememoriesco.com_September2020.json.gz,6,72,100,"{'url': 100, 'name': 100, 'offers:pricecurrenc...","{'url': 'url', 'name': 'name', 'offers:pricecu...","{'url': 'URL', 'name': 'Museum/name', 'offers:..."
1,Event,Event_davidlohmueller.com_September2020.json.gz,7,20,88,"{'startdate': 100, 'name': 100, 'description':...","{'startdate': 'startDate', 'name': 'name', 'de...","{'startdate': 'DateTime', 'name': 'Museum/name..."
2,Product,Product_winehousegvl.com_September2020.json.gz,5,43,100,"{'name': 100, 'sku': 100, 'offers:pricecurrenc...","{'name': 'name', 'sku': 'sku', 'offers:pricecu...","{'name': 'Museum/name', 'sku': 'IdentifierAT',..."
3,Product,Product_idaw.eu_September2020.json.gz,6,183,94,"{'description': 74, 'name': 100, 'image': 100,...","{'description': 'description', 'name': 'name',...","{'description': 'Text', 'name': 'Museum/name',..."
4,Product,Product_juvenileplanet.com_September2020.json.gz,5,10,88,"{'name': 100, 'sku': 80, 'offers:price': 80, '...","{'name': 'name', 'sku': 'sku', 'offers:price':...","{'name': 'Museum/name', 'sku': 'IdentifierAT',..."
...,...,...,...,...,...,...,...,...
947607,Product,Product_wolfchild.com_September2020.json.gz,6,30,100,"{'name': 100, 'description': 100, 'image': 100...","{'name': 'name', 'description': 'description',...","{'name': 'Museum/name', 'description': 'Text',..."
947608,Product,Product_gellifique.co.uk_September2020.json.gz,6,144,67,"{'name': 100, 'description': 53, 'url': 53, 'b...","{'name': 'name', 'description': 'description',...","{'name': 'Museum/name', 'description': 'Text',..."
947609,Product,Product_aransweatersdirect.com_September2020.j...,4,84,100,"{'name': 100, 'description': 100, 'offers:pric...","{'name': 'name', 'description': 'description',...","{'name': 'Museum/name', 'description': 'Text',..."
947610,Product,Product_mandystanley.com_September2020.json.gz,7,25,100,"{'name': 100, 'description': 100, 'offers:pric...","{'name': 'name', 'description': 'description',...","{'name': 'Museum/name', 'description': 'Text',..."


In [11]:
file_names = set(annotated_tables['file_name'].tolist())
len(file_names)

947612

In [13]:
#Columns to ignore in selection with a density less than 70%
ignore = []
tables_to_dict = annotated_tables.to_dict('records')
for table in tables_to_dict:
    for col in eval(table['all_cols']):
        if eval(table['all_cols'])[col] < 70:
            ignore.append(col+'_'+table['file_name'])
ignore = set(ignore)

## 1. Corner Cases Columns Selection

In [16]:
num_sim = pd.read_csv('../Similarities/subset8000_num_sim.csv.gz', compression='gzip')
num_sim2 = pd.read_csv('../Similarities/subset6000_num_sim.csv.gz', compression='gzip')
num_sim3 = pd.read_csv('../Similarities/subset6000_2_num_sim.csv.gz', compression='gzip')
date_sim = pd.read_csv('../Similarities/subset6000_datetime_sim.csv.gz', compression='gzip')
date_sim2 = pd.read_csv('../Similarities/subset7000_datetime_sim.csv.gz', compression='gzip')
date_sim3 = pd.read_csv('../Similarities/subset7000_2_datetime_sim.csv.gz', compression='gzip')
text_sim = pd.read_csv('../Similarities/subset250000_textcols_sim.csv.gz', compression='gzip')
text_sim2 = pd.read_csv('../Similarities/subset180000_textcols_sim.csv.gz', compression='gzip')
similarities = pd.concat([num_sim, num_sim2, num_sim3, date_sim, date_sim2, date_sim3, text_sim, text_sim2], ignore_index=True)
similarities['file_name'] = similarities['col_name'].apply(lambda row: row.split('_')[1]+'_'+row.split('_')[2]+'_'+row.split('_')[3])
similarities = similarities.loc[similarities['file_name'].isin(file_names)]
similarities = similarities.loc[~similarities['col_name'].isin(ignore)]
similarities

,col_name,similar_cols,file_name
0,offers:price_Product_winehousegvl.com_Septembe...,aggregaterating:reviewcount_Event_southcarolin...,Product_winehousegvl.com_September2020.json.gz
1,offers:price_Product_juvenileplanet.com_Septem...,nutrition:carbohydratecontent_Recipe_findus.se...,Product_juvenileplanet.com_September2020.json.gz
2,offers:price_Product_uppolomallets.com_Septemb...,aggregaterating:reviewcount_LocalBusiness_prom...,Product_uppolomallets.com_September2020.json.gz
3,offers:price_Product_tradefireworks.co.uk_Sept...,aggregaterating:ratingcount_Hotel_toubiz.de_Se...,Product_tradefireworks.co.uk_September2020.jso...
4,offers:price_Product_andgifts.co.uk_September2...,manufacturer_Product_lespetitslivres.com_Septe...,Product_andgifts.co.uk_September2020.json.gz
...,...,...,...
206275,address:addressregion_Person_altschools.net_Se...,address:addresscountry_LocalBusiness_tiuli.com...,Person_altschools.net_September2020.json.gz
206278,acceptsreservations_Restaurant_olieude.com_Sep...,petsallowed_Hotel_landsend.nl_September2020.js...,Restaurant_olieude.com_September2020.json.gz
206279,address:addresscountry_Restaurant_olieude.com_...,inlanguage_Movie_zone-telechargement.me_Septem...,Restaurant_olieude.com_September2020.json.gz
206280,address_LocalBusiness_ml.com_September2020.jso...,address:streetaddress_Person_lawlink.com_Septe...,LocalBusiness_ml.com_September2020.json.gz


In [ ]:
#Create a file to hold already selected columns from tables and their selection type
# Selection type: 'Missing values', 'Intra similarity', 'Inter simimilarity/dissimilarity', 'format heterogeneity', 'rare column'

### Select intra similarity columns

In [17]:
#Column -> its similar cols
intra_similarities = {}
for index, row in similarities.iterrows():
    col_name, class_name, table_name, ending = row['col_name'].split('_')
    file_name = '_'.join([class_name, table_name, ending])
    
    #If correct column
    if col_name in rel_lbls:
        
        similar_cols = row['similar_cols'].split('; ')[:10]
        intra_similarities[row['col_name']] = []

        for col in similar_cols:
            col_name2, class_name2, table_name2, ending2 = col.split('_')
            file_name_2 = '_'.join([class_name2, table_name2, ending2])

            # More than 70% of density, same table and in selected tables for CPA
            if col_name2 in rel_lbls and file_name_2 in file_names and file_name == file_name_2 and col not in ignore:
                intra_similarities[row['col_name']].append(col)

In [18]:
intra_similarities

{'offers:price_Product_winehousegvl.com_September2020.json.gz': [],
 'offers:price_Product_juvenileplanet.com_September2020.json.gz': [],
 'offers:price_Product_uppolomallets.com_September2020.json.gz': [],
 'offers:price_Product_tradefireworks.co.uk_September2020.json.gz': [],
 'offers:price_Product_andgifts.co.uk_September2020.json.gz': [],
 'offers:price_Product_ablissfullybeautifulboutique.com_September2020.json.gz': [],
 'offers:price_Product_provita.be_September2020.json.gz': [],
 'offers:price_Product_reddevildelivery.com_September2020.json.gz': [],
 'sku_Product_passionfury.com_September2020.json.gz': [],
 'offers:price_Product_grapevinyl.net_September2020.json.gz': [],
 'sku_Product_grapevinyl.net_September2020.json.gz': [],
 'sku_Product_offonlineshop.com_September2020.json.gz': [],
 'offers:price_Product_offonlineshop.com_September2020.json.gz': [],
 'offers:price_Product_vanderzee.ca_September2020.json.gz': [],
 'offers:price_Product_katwalla.com_September2020.json.gz': [],

In [19]:
#intra_sims = [ [x] + intra_similarities[x] for x in intra_similarities if len(intra_similarities[x]) > 0]

In [20]:
#intra_sims

[['productid_Product_climbing515.ru_September2020.json.gz',
  'offers:serialnumber_Product_climbing515.ru_September2020.json.gz'],
 ['offers:serialnumber_Product_climbing515.ru_September2020.json.gz',
  'productid_Product_climbing515.ru_September2020.json.gz'],
 ['identifier_Product_shoppingvirtualdabahia.com.br_September2020.json.gz',
  'productid_Product_shoppingvirtualdabahia.com.br_September2020.json.gz'],
 ['productid_Product_shoppingvirtualdabahia.com.br_September2020.json.gz',
  'identifier_Product_shoppingvirtualdabahia.com.br_September2020.json.gz'],
 ['aggregaterating:reviewcount_Recipe_noituisong.com_September2020.json.gz',
  'interactionstatistic:userinteractioncount_Recipe_noituisong.com_September2020.json.gz'],
 ['interactionstatistic:userinteractioncount_Recipe_noituisong.com_September2020.json.gz',
  'aggregaterating:reviewcount_Recipe_noituisong.com_September2020.json.gz'],
 ['gtin13_Product_pearsonelt.de_September2020.json.gz',
  'sku_Product_pearsonelt.de_September20

In [ ]:
intra_tables = {} # Mark which columns can be selected in each table
intra_cols = set() # Which columns selected overall
intra_class = {} # How many columns per schema.org type/class

for key in intra_similarities:
    if intra_similarities[key]:
        
        table = key.split('_')[1] + '_' + key.split('_')[2] + '_' + key.split('_')[3]
        class_ = key.split('_')[0]
        
        if table not in intra_tables:
            intra_tables[table] = set()
        
        if class_ not in intra_class:
            intra_class[class_] = 0
            
        intra_cols.add(key)
        intra_tables[table].add(key)
        intra_class[class_] += 1
        
        for col in intra_similarities[key]:
            intra_cols.add(col)
            intra_tables[table].add(col)
            intra_class[class_] += 1
            

In [27]:
#Selected tables
len(intra_tables)

6144

In [28]:
#Selected cols
len(set(intra_cols))

13587

In [32]:
#Mark all selected columns in a dictionary:
selected_cols_tables = {}

for table in intra_tables:
    
    selected_cols_tables[table] = {}
    
    for col in intra_tables[table]:
        selected_cols_tables[table][col.split('_')[0]] = 'Intra similarity'

In [50]:
low_class = [] #Schema.org types that have less than 1500 selected columns until now
low_label = [] #CPA labels that have less than 100 examples until now

for cl in intra_class:
    if intra_class[cl] < 1500:
        low_class.append(cl)
        
for lb in intra_labels:
    if len(intra_labels[lb]) < 100:
        low_label.append(lb)

In [80]:
low_label.remove('name')

### Select inter table similarity columns

In [37]:
inter_similarities = {}
for index, row in similarities.iterrows():
    col_name, class_name, table_name, ending = row['col_name'].split('_')
    file_name = '_'.join([class_name, table_name, ending])
    
    #if (file_name in selected_cols_tables and col_name not in selected_cols_tables[file_name]) or file_name not in selected_cols_tables:
    
    #If correct column
    if col_name in rel_lbls:
        similar_cols = row['similar_cols'].split('; ')[:10]
        inter_similarities[row['col_name']] = []

        for col in similar_cols:
            col_name2, class_name2, table_name2, ending2 = col.split('_')
            file_name_2 = '_'.join([class_name2, table_name2, ending2])

            if col_name2 in rel_lbls and file_name_2 in file_names and file_name != file_name_2 and col not in ignore:
            #if file_name_2 in file_names and file_name != file_name_2:
                #if (file_name_2 in selected_cols_tables and col_name2 not in selected_cols_tables[file_name_2]) or file_name_2 not in selected_cols_tables:
                inter_similarities[row['col_name']].append(col)

In [38]:
inter_similarities

{'offers:price_Product_winehousegvl.com_September2020.json.gz': ['offers:sku_Product_syberia.net_September2020.json.gz'],
 'offers:price_Product_juvenileplanet.com_September2020.json.gz': ['nutrition:carbohydratecontent_Recipe_moderncook.com.vn_September2020.json.gz',
  'aggregaterating:reviewcount_Restaurant_food94.com_September2020.json.gz',
  'aggregaterating:reviewcount_Hotel_toptours.hu_September2020.json.gz',
  'geo:latitude_Museum_trekraw.com_September2020.json.gz'],
 'offers:price_Product_uppolomallets.com_September2020.json.gz': ['aggregaterating:reviewcount_LocalBusiness_promobil.de_September2020.json.gz',
  'nutrition:cholesterolcontent_Recipe_faylinameir.com_September2020.json.gz',
  'offers:mpn_Product_cyclony.ru_September2020.json.gz',
  'category_Product_calcuttboatsshop.com_September2020.json.gz',
  'nutrition:cholesterolcontent_Recipe_diabeticdinnerrecipes.com_September2020.json.gz',
  'nutrition:carbohydratecontent_Recipe_faylinameir.com_September2020.json.gz'],
 'off

In [39]:
inter_sims = [ [x] + inter_similarities[x] for x in inter_similarities if len(inter_similarities[x]) > 0]

In [ ]:
# Selecting inter similarities: 
#Count how many columns for low class number of columns and for low CTA labels can be selected
select_test = []
i = 0

for cols in inter_sims:
    select_test.append([])
    
    for col in cols:
        class_ = col.split('_')[1]
        
        #If table is in low class and in low label
        if class_ in low_class and rel_lbls[col.split('_')[0]] in low_label:
            #If it hasn't been selected already in the intra similarity phase
            if (table in selected_cols_tables and col.split('_')[0] not in selected_cols_tables[table]) or table not in selected_cols_tables:
                select_test[i].append(col)
    i+=1
    
    
# And Filter out columns with less than 1 similar column
s = [ x for x in select_test if len(x) > 1]


# And Select maximum 3500 cols for each CPA label
sel_labels = {}

for sim in s:
    for col in sim:
        
        if rel_lbls[col.split('_')[0]] not in sel_labels:
            sel_labels[rel_lbls[col.split('_')[0]]] = []
        
        if len(sel_labels[type_lbls[col]]) < 3500:
            sel_labels[rel_lbls[col.split('_')[0]]].append(col)

sel_labels

In [323]:
#Select columns which are included in sel_labels
selected_test_2 = []
i = 0

for cols in inter_sims:
    selected_test_2.append([])
    
    for col in cols:

        if rel_lbls[col.split('_')[0]] in sel_labels_2 and col in sel_labels_2[rel_lbls[col.split('_')[0]]]:
            selected_test_2[i].append(col)

    i+=1
    
#Filter out columns with no similar columns
s_2 = [ x for x in selected_test_2 if len(x) > 1]

In [330]:
# Add the new selected columns to the already selected ones
selected_cols = copy.deepcopy(selected_cols_tables)

for s in s_2:
    for col in s:
        tab = col.split('_')[1] + '_' + col.split('_')[2] + '_' + col.split('_')[3]
        
        if tab not in sel_columns_2:
            sel_columns_2[tab] = {}
        
        sel_columns_2[tab][col.split('_')[0]] = 'Inter Similarity'
        

## 2. Select Missing values columns

In [334]:
low_dens_70 = annotated_tables.loc[ (annotated_tables['overall_table_density'] < 70) ]
low_dens_70

,class,file_name,column_count,number_of_rows,overall_table_density,all_cols,rel_labels,type_labels
13,LocalBusiness,LocalBusiness_litmind.com_September2020.json.gz,9,63,47,"{'telephone': 87, 'name': 100, 'address:addres...","{'telephone': 'telephone', 'name': 'name', 'ad...","{'telephone': 'telephone', 'name': 'Museum/nam..."
50,Product,Product_internetprodavnice.rs_September2020.js...,10,39,63,"{'description': 74, 'name': 100, 'url': 74, 'o...","{'description': 'description', 'name': 'name',...","{'description': 'Text', 'name': 'Museum/name',..."
236,Product,Product_tissusprovencaux.fr_September2020.json.gz,13,17,39,"{'name': 100, 'description': 23, 'sku': 5, 'br...","{'name': 'name', 'description': 'description',...","{'name': 'Museum/name', 'description': 'Text',..."
240,Product,Product_budomaster.ru_September2020.json.gz,10,40,62,"{'description': 100, 'name': 100, 'sku': 100, ...","{'description': 'description', 'name': 'name',...","{'description': 'Text', 'name': 'Museum/name',..."
300,Event,Event_ncstatecollege.edu_September2020.json.gz,14,103,61,"{'startdate': 100, 'name': 100, 'description':...","{'startdate': 'startDate', 'name': 'name', 'de...","{'startdate': 'DateTime', 'name': 'Museum/name..."
...,...,...,...,...,...,...,...,...
947346,Product,Product_rovuxfootwear.com_September2020.json.gz,9,124,66,"{'name': 100, 'description': 58, 'offers:price...","{'name': 'name', 'description': 'description',...","{'name': 'Museum/name', 'description': 'Text',..."
947361,Product,Product_farmaciamarket.es_September2020.json.gz,11,344,69,"{'name': 100, 'description': 34, 'url': 21, 'b...","{'name': 'name', 'description': 'description',...","{'name': 'Museum/name', 'description': 'Text',..."
947481,Recipe,Recipe_herbncowgirl.com_September2020.json.gz,10,19,62,"{'name': 94, 'recipeingredient': 68, 'datepubl...","{'name': 'name', 'recipeingredient': 'recipeIn...","{'name': 'Museum/name', 'recipeingredient': 'T..."
947542,Product,Product_sibpech.ru_September2020.json.gz,10,14,67,"{'name': 100, 'description': 7, 'offers:pricec...","{'name': 'name', 'description': 'description',...","{'name': 'Museum/name', 'description': 'Text',..."


In [335]:
#How many low density columns can be annotated per table
low_dens_70['low_cols'] = low_dens_70['all_cols'].apply(lambda row: len( [x for x in eval(row) if eval(row)[x] < 70 and eval(row)[x] > 10 ] ))

/tmp/ipykernel_767103/755326462.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low_dens_70['low_cols'] = low_dens_70['all_cols'].apply(lambda row: len( [x for x in eval(row) if eval(row)[x] < 70 and eval(row)[x] > 10 ] ))


In [338]:
low_dens_70.loc[ (low_dens_70['low_cols'] >= 3) ].groupby(['class'])['low_cols'].sum()

class
Book                632
CreativeWork        616
Event              7426
Hotel               664
JobPosting           37
LocalBusiness      2015
Movie              1816
MusicAlbum           13
MusicRecording       35
Person             1299
Place               545
Product           37625
Recipe            11074
Restaurant          329
SportsEvent          54
TVEpisode            31
Name: low_cols, dtype: int64

In [372]:
# Limit Product, Recipe and Event tables to 800
prods = low_dens_70.loc[ (low_dens_70['low_cols'] >= 3) & (low_dens_70['class'] == 'Product' ) ][800:]
recipe = low_dens_70.loc[ (low_dens_70['low_cols'] >= 3) & (low_dens_70['class'] == 'Recipe' ) ][800:]
event = low_dens_70.loc[ (low_dens_70['low_cols'] >= 3) & (low_dens_70['class'] == 'Event' ) ][800:]

remove_tables = prods['file_name'].tolist() + recipe['file_name'].tolist() + event['file_name'].tolist()

In [375]:
#Select columns with low density from the low density tables that have at least 3 low density columns
#Add to all other selected columns

for index, row in low_dens_70.loc[ (low_dens_70['low_cols'] >= 3) ].iterrows():
    file_name = row['file_name']
    
    if file_name not in remove_tables:    
        if file_name not in selected_cols:
            selected_cols[file_name] = {}

        #Look at low density columns and select if not already annotated from intra sim
        annotated_cols = eval(row['all_cols'])

        for column in annotated_cols:
            if annotated_cols[column] < 70 and annotated_cols[column] > 10 and column not in selected_cols[file_name]:
                selected_cols[file_name][column] = 'Missing values'


In [ ]:
#Save selected columns to file
selection = annotated_tables.loc[ annotated_tables['file_name'].isin(selected_cols) ]
sel_cols = []
for index, row in selection.iterrows():
    sel_cols.append(selected_cols[row['file_name']])
selection['selected_cols'] = sel_cols
selection.to_csv('output-data/cpa-datasets/selected_1.csv', index=False)